In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import argparse
import trfl
from utility_v2 import pad_history, calculate_hit


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
epoch = 30
data = r'E:\MIni\v2\data'
batch_size = 256
hidden_factor = 64
r_click = 0.2
r_buy = 1
lr = 0.01

In [3]:
class GRUnetwork(tf.keras.Model):
    def __init__(self, hidden_size, item_num):
        super(GRUnetwork, self).__init__(name='GRUnetwork')

        self.hidden_size = hidden_size
        self.item_num = int(item_num)

        self.state_embeddings = tf.keras.layers.Embedding(self.item_num+1, self.hidden_size, name='state_embeddings')
        self.gru_layer = tf.keras.layers.GRU(self.hidden_size)
        self.output_layer = tf.keras.layers.Dense(self.item_num, activation=None, name='fc')

    def call(self, inputs):
        
        inputs = tf.convert_to_tensor(inputs)
        input_emb = self.state_embeddings(inputs)

        states_hidden = self.gru_layer(input_emb, initial_state=None)

        output = self.output_layer(states_hidden)

        return output


In [13]:
def evaluate(GRUnet):
    eval_sessions = pd.read_pickle(os.path.join(data_directory, 'sampled_val.df'))
    eval_ids = eval_sessions.session_id.unique()
    groups = eval_sessions.groupby('session_id')
    batch = 100
    evaluated = 0
    total_clicks = 0.0
    total_purchase = 0.0
    total_reward = [0, 0, 0, 0]
    hit_clicks = [0, 0, 0, 0]
    ndcg_clicks = [0, 0, 0, 0]
    hit_purchase = [0, 0, 0, 0]
    ndcg_purchase = [0, 0, 0, 0]
    while evaluated < len(eval_ids):
        states, len_states, actions, rewards = [], [], [], []
        for i in range(batch):
            id = eval_ids[evaluated]
            group = groups.get_group(id)
            history = []
            for index, row in group.iterrows():
                state = list(history)
                len_states.append(state_size if len(state) >= state_size else 1 if len(state) == 0 else len(state))
                state = pad_history(state, state_size, item_num)
                states.append(state)
                action = row['item_id']
                is_buy = row['is_buy']
                reward = reward_buy if is_buy == 1 else reward_click
                if is_buy == 1:
                    total_purchase += 1.0
                else:
                    total_clicks += 1.0
                actions.append(action)
                rewards.append(reward)
                history.append(row['item_id'])
            evaluated += 1
        prediction = GRUnet(states)
        sorted_list = np.argsort(prediction)
        calculate_hit(sorted_list, topk, actions, rewards, reward_click, total_reward, hit_clicks, ndcg_clicks, hit_purchase, ndcg_purchase)
    print('#############################################################')
    print('total clicks: %d, total purchase:%d' % (total_clicks, total_purchase))
    for i in range(len(topk)):
        hr_click = hit_clicks[i] / total_clicks
        hr_purchase = hit_purchase[i] / total_purchase
        ng_click = ndcg_clicks[i] / total_clicks
        ng_purchase = ndcg_purchase[i] / total_purchase
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        print('cumulative reward @ %d: %f' % (topk[i], total_reward[i]))
        print('clicks hr ndcg @ %d : %f, %f' % (topk[i], hr_click, ng_click))
        print('purchase hr and ndcg @%d : %f, %f' % (topk[i], hr_purchase, ng_purchase))
    print('#############################################################')


In [5]:
data_directory = data
data_statis = pd.read_pickle(
os.path.join(data_directory, 'data_statis.df'))  # read data statistics, includeing state_size and item_num
state_size = data_statis['state_size'][0]  # the length of history to define the state
item_num = data_statis['item_num'][0]  # total number of items
reward_click = r_click
reward_buy = r_buy
topk = [5, 10, 15, 20]

In [6]:
GRUnet = GRUnetwork(hidden_size=hidden_factor, item_num=item_num)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)


In [15]:
replay_buffer = pd.read_pickle(os.path.join(data_directory, 'replay_buffer.df'))
total_step = 0

num_rows=replay_buffer.shape[0]
num_batches=int(num_rows/batch_size)

for i in range(epoch):
    for j in range(num_batches):

        batch = replay_buffer.sample(n=batch_size).to_dict()
        
        state = list(batch['state'].values())
        len_state = list(batch['len_state'].values())
        target = list(batch['action'].values())

        with tf.GradientTape() as tape:
            logits = GRUnet(state)
            
            loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=target, logits=logits))

        gradients = tape.gradient(loss, GRUnet.trainable_variables)
        optimizer.apply_gradients(zip(gradients, GRUnet.trainable_variables))

        
        total_step += 1
        if total_step % 200 == 0:
            print("the loss in %dth batch is: %f" % (total_step, loss))
        if total_step % 2000 == 0:
            evaluate(GRUnet)



KeyboardInterrupt: 

: 